# Análisis de datos con Python y MongoDB

La información del dataset proporcionada por el departamento Policial de San Francisco, se encuentra abierta al público y contiene datos de todas las incidencias generadas por actividades criminales producidas desde el año 2003 hasta la actualidad. Usaremos la base de datos datascience que hemos creado en MongoDB con la coleccion incidents.

In [14]:
%config IPCompleter.greedy=True

%matplotlib inline
%config InlineBackend.figure_format='svg'
from IPython.display import display,HTML
import pandas as pd
import seaborn as sns
from scipy.stats import kendalltau
import numpy as np
import math
import matplotlib.pyplot as plt

from prettypandas import PrettyPandas
sns.set(style="ticks")
sns.set_context(context="notebook",font_scale=1)

import string
import tqdm # a cool progress bar
import re
import json

import pymongo
from pymongo import MongoClient

In [16]:
########################################################### Database Connection and Load ############################
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.datascience
collection = db.incidents

Mongo version 3.4.0


Calculamos cuantos documentos tenemos en la coleccion

In [18]:
d = db.incidents.count()
d

2186988

In [19]:
#Check if you can access the data from the MongoDB.
cursor = collection.find().sort('sex',pymongo.ASCENDING).limit(1)
for doc in cursor:
    print(doc)

{'_id': ObjectId('5ac48f27a2eb3a9495192d44'), 'IncidntNum': 150060275, 'Category': 'NON-CRIMINAL', 'Descript': 'LOST PROPERTY', 'DayOfWeek': 'Monday', 'Date': '01/19/2015', 'Time': '14:00', 'PdDistrict': 'MISSION', 'Resolution': 'NONE', 'Address': '18TH ST / VALENCIA ST', 'X': -122.42158168137, 'Y': 37.7617007179518, 'Location': '(37.7617007179518, -122.42158168137)', 'PdId': 15006027571000}


In [20]:
pipeline = [
        {"$match": {"Category":"ROBBERY"}},
]

aggResult = collection.aggregate(pipeline)
robbery = pd.DataFrame(list(aggResult))
robbery.head()

,Address,Category,Date,DayOfWeek,Descript,IncidntNum,Location,PdDistrict,PdId,Resolution,Time,X,Y,_id
0,300 Block of LEAVENWORTH ST,ROBBERY,02/01/2015,Sunday,"ROBBERY, BODILY FORCE",150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821003074,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d45
1,2200 Block of MARKET ST,ROBBERY,02/01/2015,Sunday,"ROBBERY, ARMED WITH A KNIFE",150098367,"(37.7651107322703, -122.432198022433)",MISSION,15009836703072,NONE,16:20,-122.432198,37.765111,5ac48f27a2eb3a9495192d52
2,PACIFIC AV / GRANT AV,ROBBERY,02/01/2015,Sunday,"ROBBERY ON THE STREET, STRONGARM",150098414,"(37.7969028838908, -122.406831986427)",CENTRAL,15009841403014,"ARREST, BOOKED",17:05,-122.406832,37.796903,5ac48f27a2eb3a9495192d58
3,PACIFIC AV / GRANT AV,ROBBERY,02/01/2015,Sunday,"ROBBERY, BODILY FORCE",150098414,"(37.7969028838908, -122.406831986427)",CENTRAL,15009841403074,"ARREST, BOOKED",17:05,-122.406832,37.796903,5ac48f27a2eb3a9495192d59
4,400 Block of ELLIS ST,ROBBERY,02/01/2015,Sunday,ATTEMPTED ROBBERY WITH A DEADLY WEAPON,150098420,"(37.784696907904, -122.413609328985)",TENDERLOIN,15009842003473,"ARREST, BOOKED",17:10,-122.413609,37.784697,5ac48f27a2eb3a9495192d5d


In [26]:
pipeline = [
        {"$match": {"Category":"ASSAULT"}},
]

aggResult = collection.aggregate(pipeline)
assault = pd.DataFrame(list(aggResult))
assault.head()

,Address,Category,Date,DayOfWeek,Descript,IncidntNum,Location,PdDistrict,PdId,Resolution,Time,X,Y,_id
0,300 Block of LEAVENWORTH ST,ASSAULT,02/01/2015,Sunday,AGGRAVATED ASSAULT WITH BODILY FORCE,150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821004014,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d46
1,PACIFIC AV / GRANT AV,ASSAULT,02/01/2015,Sunday,AGGRAVATED ASSAULT WITH BODILY FORCE,150098414,"(37.7969028838908, -122.406831986427)",CENTRAL,15009841404014,"ARREST, BOOKED",17:05,-122.406832,37.796903,5ac48f27a2eb3a9495192d5b
2,PACIFIC AV / GRANT AV,ASSAULT,02/01/2015,Sunday,BATTERY WITH SERIOUS INJURIES,150098414,"(37.7969028838908, -122.406831986427)",CENTRAL,15009841404136,"ARREST, BOOKED",17:05,-122.406832,37.796903,5ac48f27a2eb3a9495192d5c
3,400 Block of ELLIS ST,ASSAULT,02/01/2015,Sunday,AGGRAVATED ASSAULT WITH BODILY FORCE,150098420,"(37.784696907904, -122.413609328985)",TENDERLOIN,15009842004014,"ARREST, BOOKED",17:10,-122.413609,37.784697,5ac48f27a2eb3a9495192d5e
4,2000 Block of MISSION ST,ASSAULT,02/01/2015,Sunday,BATTERY OF A POLICE OFFICER,150098458,"(37.764228935718, -122.419520367886)",MISSION,15009845804154,"ARREST, BOOKED",16:56,-122.419520,37.764229,5ac48f27a2eb3a9495192d62


In [27]:
pipeline = [
        {"$match": {"Category":"DRUG/NARCOTIC"}},
]

aggResult = collection.aggregate(pipeline)
drug = pd.DataFrame(list(aggResult))
drug.head()

,Address,Category,Date,DayOfWeek,Descript,IncidntNum,Location,PdDistrict,PdId,Resolution,Time,X,Y,_id
0,1700 Block of HARRISON ST,DRUG/NARCOTIC,02/01/2015,Sunday,POSSESSION OF METH-AMPHETAMINE,150098345,"(37.7690748003847, -122.413354187018)",MISSION,15009834516650,"ARREST, BOOKED",14:00,-122.413354,37.769075,5ac48f27a2eb3a9495192d4f
1,1700 Block of HARRISON ST,DRUG/NARCOTIC,02/01/2015,Sunday,POSSESSION OF NARCOTICS PARAPHERNALIA,150098345,"(37.7690748003847, -122.413354187018)",MISSION,15009834516710,"ARREST, BOOKED",14:00,-122.413354,37.769075,5ac48f27a2eb3a9495192d50
2,2000 Block of MISSION ST,DRUG/NARCOTIC,02/01/2015,Sunday,POSSESSION OF BASE/ROCK COCAINE FOR SALE,150098458,"(37.764228935718, -122.419520367886)",MISSION,15009845816623,"ARREST, BOOKED",16:56,-122.419520,37.764229,5ac48f27a2eb3a9495192d64
3,MISSION ST / 15TH ST,DRUG/NARCOTIC,02/01/2015,Sunday,POSSESSION OF METH-AMPHETAMINE,150098527,"(37.7666737551835, -122.419827929961)",MISSION,15009852716650,"ARREST, BOOKED",17:02,-122.419828,37.766674,5ac48f27a2eb3a9495192d6e
4,700 Block of MARKET ST,DRUG/NARCOTIC,02/01/2015,Sunday,POSSESSION OF MARIJUANA,150098997,"(37.7871160984672, -122.403919148357)",SOUTHERN,15009899716010,NONE,20:35,-122.403919,37.787116,5ac48f27a2eb3a9495192da7


### Averiguar que tipos de resoluciones existen en nuestros datos

In [34]:
db.incidents.distinct( "Resolution" )

['NONE',
 'ARREST, BOOKED',
 'EXCEPTIONAL CLEARANCE',
 'ARREST, CITED',
 'UNFOUNDED',
 'JUVENILE BOOKED',
 'CLEARED-CONTACT JUVENILE FOR MORE INFO',
 'PSYCHOPATHIC CASE',
 'LOCATED',
 'JUVENILE ADMONISHED',
 'COMPLAINANT REFUSES TO PROSECUTE',
 'PROSECUTED BY OUTSIDE AGENCY',
 'NOT PROSECUTED',
 'JUVENILE CITED',
 'JUVENILE DIVERTED',
 'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
 'PROSECUTED FOR LESSER OFFENSE']

### Distribucion de Categories

In [30]:
print("{} robberies ({:.1%}), {} assaults ({:.1%}), {} drugs ({:.1%})".format(
    len(robbery),len(robbery)/d,
    len(assault),len(assault)/d,
    len(drug),len(drug)/d))

55242 robberies (2.5%), 191952 assaults (8.8%), 118739 drugs (5.4%)


In [94]:
from pprint import pprint
cursor = collection.find().sort('Category',pymongo.ASCENDING).limit(10)
for doc in cursor:
    pprint(doc)

{'Address': 'SACRAMENTO ST / POLK ST',
 'Category': 'ARSON',
 'Date': '01/04/2014',
 'DayOfWeek': 'Saturday',
 'Descript': 'ARSON',
 'IncidntNum': 140009459,
 'Location': '(37.7914943051906, -122.420874632415)',
 'PdDistrict': 'NORTHERN',
 'PdId': 14000945926030,
 'Resolution': 'ARREST, BOOKED',
 'Time': '03:52',
 'X': -122.420874632415,
 'Y': 37.7914943051906,
 '_id': ObjectId('5ac48f27a2eb3a9495192d63')}
{'Address': '500 Block of VALENCIA ST',
 'Category': 'ARSON',
 'Date': '02/02/2015',
 'DayOfWeek': 'Monday',
 'Descript': 'ARSON OF AN INHABITED DWELLING',
 'IncidntNum': 150100081,
 'Location': '(37.7640888944532, -122.421876488492)',
 'PdDistrict': 'MISSION',
 'PdId': 15010008126036,
 'Resolution': 'ARREST, BOOKED',
 'Time': '10:05',
 'X': -122.421876488492,
 'Y': 37.7640888944532,
 '_id': ObjectId('5ac48f27a2eb3a9495192e25')}
{'Address': '200 Block of SHOTWELL ST',
 'Category': 'ARSON',
 'Date': '02/02/2015',
 'DayOfWeek': 'Monday',
 'Descript': 'ARSON OF A VEHICLE',
 'IncidntNum'

In [35]:
# aislar los dias Sunday
aggResult = collection.aggregate([{"$match": {"DayOfWeek":"Sunday"}}])
sunday = pd.DataFrame(list(aggResult))

In [36]:
# aislar los dias Saturday 
aggResult = collection.aggregate([{"$match": {"DayOfWeek":"Saturday"}}])
saturday = pd.DataFrame(list(aggResult))

In [37]:
print("{} Domingos ({:.1%}), {} Sabados ({:.1%})".format(
    len(sunday),len(sunday)/d,
    len(saturday),len(saturday)/d))

290936 Domingos (13.3%), 316451 Sabados (14.5%)


## Consultas varias

#### Número de incidencias en Sunday

In [46]:
db.incidents.count({"DayOfWeek":"Sunday"})

290936

#### Incidencias en el dia 02/01/2015

In [57]:
pipeline = [
        {"$match": {"Date":"02/01/2015"}},
]

aggResult = collection.aggregate(pipeline)
dia = pd.DataFrame(list(aggResult))
dia.head()

,Address,Category,Date,DayOfWeek,Descript,IncidntNum,Location,PdDistrict,PdId,Resolution,Time,X,Y,_id
0,300 Block of LEAVENWORTH ST,ROBBERY,02/01/2015,Sunday,"ROBBERY, BODILY FORCE",150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821003074,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d45
1,300 Block of LEAVENWORTH ST,ASSAULT,02/01/2015,Sunday,AGGRAVATED ASSAULT WITH BODILY FORCE,150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821004014,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d46
2,300 Block of LEAVENWORTH ST,SECONDARY CODES,02/01/2015,Sunday,DOMESTIC VIOLENCE,150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821015200,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d47
3,400 Block of LOCUST ST,NON-CRIMINAL,02/01/2015,Sunday,AIDED CASE -PROPERTY FOR DESTRUCTION,150098232,"(37.7870853907529, -122.451781767894)",RICHMOND,15009823251041,NONE,16:21,-122.451782,37.787085,5ac48f27a2eb3a9495192d49
4,1700 Block of HARRISON ST,LARCENY/THEFT,02/01/2015,Sunday,PETTY THEFT SHOPLIFTING,150098345,"(37.7690748003847, -122.413354187018)",MISSION,15009834506362,"ARREST, BOOKED",14:00,-122.413354,37.769075,5ac48f27a2eb3a9495192d4e


#### Número de incidencias el 02/01/2015

In [59]:
db.incidents.count({"Date":"02/01/2015"})

466

#### Incidencias en la Zona de LEAVENWORTH

In [78]:
pipeline = [
        {'$match': {'Address':{'$regex': 'LEAVENWORTH'}}},
]

aggResult = collection.aggregate(pipeline)
zona = pd.DataFrame(list(aggResult))
zona.head()

,Address,Category,Date,DayOfWeek,Descript,IncidntNum,Location,PdDistrict,PdId,Resolution,Time,X,Y,_id
0,300 Block of LEAVENWORTH ST,ROBBERY,02/01/2015,Sunday,"ROBBERY, BODILY FORCE",150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821003074,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d45
1,300 Block of LEAVENWORTH ST,ASSAULT,02/01/2015,Sunday,AGGRAVATED ASSAULT WITH BODILY FORCE,150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821004014,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d46
2,300 Block of LEAVENWORTH ST,SECONDARY CODES,02/01/2015,Sunday,DOMESTIC VIOLENCE,150098210,"(37.7841907151119, -122.414406029855)",TENDERLOIN,15009821015200,NONE,15:45,-122.414406,37.784191,5ac48f27a2eb3a9495192d47
3,MCALLISTER ST / LEAVENWORTH ST,RECOVERED VEHICLE,02/01/2015,Sunday,"VEHICLE, RECOVERED, AUTO",150098919,"(37.7809258336852, -122.413679376888)",TENDERLOIN,15009891907041,JUVENILE BOOKED,19:53,-122.413679,37.780926,5ac48f27a2eb3a9495192d99
4,MCALLISTER ST / LEAVENWORTH ST,OTHER OFFENSES,02/01/2015,Sunday,CONTRIBUTING TO THE DELINQUENCY OF MINOR,150098919,"(37.7809258336852, -122.413679376888)",TENDERLOIN,15009891915030,JUVENILE BOOKED,19:53,-122.413679,37.780926,5ac48f27a2eb3a9495192d9a


#### Actividad criminal por tipo de delito, por año/dia (por tipo de delito)

In [79]:
pipeline = [
        {'$match': {'Date':{'$regex': '2013'},'Category':'ROBBERY'}},
]

aggResult = collection.aggregate(pipeline)
act = pd.DataFrame(list(aggResult))
act.head()

,Address,Category,Date,DayOfWeek,Descript,IncidntNum,Location,PdDistrict,PdId,Resolution,Time,X,Y,_id
0,1600 Block of KIRKWOOD AV,ROBBERY,02/10/2013,Sunday,"ROBBERY, ARMED WITH A KNIFE",130117084,"(37.7386625599684, -122.390952930587)",BAYVIEW,13011708403072,NONE,01:46,-122.390953,37.738663,5ac48f2fa2eb3a94951ce496
1,2200 Block of GEARY BL,ROBBERY,09/18/2013,Wednesday,"ROBBERY, BODILY FORCE",130790232,"(37.7833242481047, -122.440341074545)",PARK,13079023203074,NONE,23:00,-122.440341,37.783324,5ac48f2fa2eb3a94951ce73c
2,FULTON ST / BAKER ST,ROBBERY,02/27/2013,Wednesday,ROBBERY ON THE STREET WITH A GUN,130168366,"(37.7764331716134, -122.441488426414)",PARK,13016836603011,NONE,02:32,-122.441488,37.776433,5ac48f2fa2eb3a94951ce804
3,MARKET ST / CASTRO ST,ROBBERY,03/15/2013,Friday,"ROBBERY, BODILY FORCE",130215296,"(37.7626702770872, -122.435187699349)",MISSION,13021529603074,NONE,03:15,-122.435188,37.762670,5ac48f2fa2eb3a94951cea67
4,HAYES ST / DIVISADERO ST,ROBBERY,09/26/2013,Thursday,ATTEMPTED ROBBERY WITH A GUN,130810830,"(37.7749912944366, -122.437799703468)",PARK,13081083003471,NONE,07:15,-122.437800,37.774991,5ac48f2fa2eb3a94951ceac5


### Una pequeña función para contar el número de documentos según una colección dada.

In [85]:
def mongo_stats(mg_coll, filter={}):
    try:
        # connect to database
        client = MongoClient('localhost', 27017)
        db = client.datascience
        
        coll = db[mg_coll]
        return coll.find(filter).count()
    except:
        return False

In [86]:
mongo_stats("incidents")

2186988

### O una función para obtener el último documento por fecha:

In [87]:
def get_last_doc(mg_coll, filter = {}, query_limit = 1):
    try:
        # connect to database
        client = MongoClient('localhost', 27017)
        db = client.datascience
        
        coll = db[mg_coll]
        cursor = coll.find(filter, limit=query_limit).sort('date',pymongo.ASCENDING)
        return cursor
 
    except Exception as e:
        print ("No se pudo conectar a la base de datos: ", e)

In [92]:
from pprint import pprint
for doc in get_last_doc("incidents"):
    pprint(doc)

{'Address': '18TH ST / VALENCIA ST',
 'Category': 'NON-CRIMINAL',
 'Date': '01/19/2015',
 'DayOfWeek': 'Monday',
 'Descript': 'LOST PROPERTY',
 'IncidntNum': 150060275,
 'Location': '(37.7617007179518, -122.42158168137)',
 'PdDistrict': 'MISSION',
 'PdId': 15006027571000,
 'Resolution': 'NONE',
 'Time': '14:00',
 'X': -122.42158168137,
 'Y': 37.7617007179518,
 '_id': ObjectId('5ac48f27a2eb3a9495192d44')}
